# 📘 MyAiMeanVariance_v5a_fullstack 策略說明

本 Notebook 示範如何從 ETF 成分股中建立 AI 預測強化的 Mean-Variance 投資組合。

## 策略流程圖：
```
ETF成分股(0050+0056) 
    ↓
GPT μ 預測 (基本面 + 新聞)
    ↓
動能修正（近 60 日趨勢）
    ↓
信心懲罰（波動越大 μ 打折）
    ↓
Mean-Variance 最佳化 + L2 正則化
    ↓
動態風控（VIX + JNK + Drawdown）
    ↓
輸出持股權重 + Top5 說明
```

In [ ]:
# ✅ 匯入主要模組與資料
from MyAiMeanVariance_v5a_fullstack import (
    fetch_0050_components, fetch_0056_components,
    gpt_contextual_rating,
    apply_momentum_adjustment, apply_mu_confidence_adjustment,
    optimize_portfolio, apply_risk_controls,
    load_prices, sample_cov
)
import yfinance as yf
import pandas as pd
from datetime import datetime, timedelta

In [ ]:
# ▓ STEP 1：取得個股名單（0050 + 0056）並排除純金融股
c0050 = fetch_0050_components()
c0056 = fetch_0056_components()
tickers = sorted(set([code for code, _ in c0050 + c0056 if not code.startswith("289")]))
tickers[:10]  # 顯示前 10 檔股票代號

In [ ]:
# ▓ STEP 2：下載價格資料與歷史共變異數
end_date = datetime.today()
start_date = end_date - timedelta(days=365)

prices = load_prices(tickers, start_date.strftime('%Y-%m-%d'), end_date.strftime('%Y-%m-%d'))
cov_matrix = sample_cov(prices)
prices.tail(3)

In [ ]:
# ▓ STEP 3：AI μ 預測 → 動能修正 → 信心調整
mu_base = gpt_contextual_rating(tickers, horizon_months=3)
mu_momentum = apply_momentum_adjustment(mu_base, prices)
mu_final = apply_mu_confidence_adjustment(mu_momentum, prices)
mu_final.sort_values(ascending=False).head(10)

In [ ]:
# ▓ STEP 4：最佳化投資組合（L2 + Max Sharpe）
from pypfopt.risk_models import sample_cov

weights_raw, (exp_ret, vol, sharpe) = optimize_portfolio(
    mu_final, cov_matrix, risk_free_rate=0.015, regularize=True, gamma=0.1
)
pd.Series(weights_raw).sort_values(ascending=False).head(10)

In [ ]:
# ▓ STEP 5：應用風控（風險訊號 + 權重上限）
weights_final = apply_risk_controls(weights_raw, {}, prices)
pd.Series(weights_final).sort_values(ascending=False).head(10)

## 🔍 Top 持股說明（μ + 產業）

In [ ]:
for tk, w in sorted(weights_final.items(), key=lambda x: -x[1])[:5]:
    try:
        info = yf.Ticker(tk).info
        print(f"{tk}: {w:.2%} | μ={mu_final[tk]:.2%} | Sector={info.get('sector', 'N/A')}")
    except Exception as e:
        print(f"{tk}: {w:.2%} | μ={mu_final[tk]:.2%} | ⚠️ Sector info unavailable")